# Detecting motion of Vehicles

#  

#### importing libraries

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
cap = cv2.VideoCapture("input/input_video.mp4")

# UNCOMMENT THE NEXT LINE TO SAVE THE VIDEO --------------------------------------------------------------------
writer = cv2.VideoWriter('output/result.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 22, (1280,720))  


# reading first frame
ret, frame1 = cap.read() 

while cap.isOpened():
    flag = False
    ret, frame2 = cap.read()
    if not ret:
        break
    
    
    # absolute difference between two consecutive frame
    diff = cv2.absdiff(frame1, frame2)
    
    
    # gray scaling image
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    
    blur = cv2.GaussianBlur(gray, (3,3), 12)

    ret, thresh = cv2.threshold(blur, 34, 255, cv2.THRESH_BINARY)
    
    
    # applying 3 sets of erode-dialate pairs to grab the fine details
    
    # the follownig filters have different kernel size, iterations
    
    # these were varied in wide ranges to find the optimal value so as to extract best information from the video.
    # these values may not apply to every video and might require some tweaking
    
    # this lacking of generalized output is acceptable since we are not using any classifier or trained model to detect vehicles.
    # we are just using change in pixel values
    
    # the outputs varys with weather conditions at that time(vehicle shadows interferes), angle at which the video is taken, heavy or light traffic 
    # as in heavy traffic, vehicles tend to merge into one another
    
    
    #  erode(4,4)*1 --> dialate(8,8)*1 --> erode(7,7)*1 --> dialate(7,7)*1 --> erode(9,9)*1 --> dialate(9,9)*2
    
    kernel = np.ones((4,4),dtype=np.uint8)
    erode1 = cv2.erode(thresh, kernel, iterations = 1)
    
    kernel = np.ones((8,8),dtype=np.uint8)
    dialate1 = cv2.dilate(erode1,kernel,iterations = 1)
    
    
    
    kernel = np.ones((7,7),dtype=np.uint8)
    erode2 = cv2.erode(dialate1, kernel, iterations = 1)
    
    kernel = np.ones((7,7),dtype=np.uint8)
    dialate2 = cv2.dilate(erode2,kernel,iterations = 1)

    
    
    kernel = np.ones((9,9),dtype=np.uint8)
    erode3 = cv2.erode(dialate2, kernel, iterations = 1)
    
    kernel = np.ones((9,9),dtype=np.uint8)
    dialate3 = cv2.dilate(erode3,kernel,iterations = 2)
    
    
    
    # extracting external contours
    contours, hierarchy = cv2.findContours(dialate3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    final = frame2.copy()
    for cnt in contours:
        flag = True
        (x,y,w,h) = cv2.boundingRect(cnt)
        cv2.rectangle(final, (x, y), (x+w, y+h), (12,232,21), 2)
    
    
    cv2.imshow('frames', frame2)
    if flag:
        cv2.imshow('final', final)
        # UNCOMMENT THE NEXT LINE TO SAVE THE VIDEO --------------------------------------------------------------------
        writer.write(final)
        

    
    frame1 = frame2
    
    
    if cv2.waitKey(1) & 0xFF == 27:
        break


# UNCOMMENT THE NEXT LINE TO SAVE THE VIDEO --------------------------------------------------------------------
writer.release()
cap.release()
cv2.destroyAllWindows()